In [ ]:
import matplotlib.pyplot as plt
import random
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import os
# print(os.listdir('/home'))
!pip install ../input/image-classify/image_classifiers-1.0.0-py3-none-any.whl
!pip install ../input/efficient/efficientnet-master/
!pip install ../input/sm-model/segmentation_models-master/
import efficientnet.keras as efn
from keras.layers import GlobalAvgPool2D,Dense,Dropout
from keras.models import Model

model = efn.EfficientNetB0(include_top=False,input_shape=(128,800,3),weights=None)
pool = GlobalAvgPool2D()(model.output)
output = Dropout(0.5)(pool)
output = Dense(4,activation='sigmoid')(output)
model_cls = Model(model.input,output)
model_cls.summary()
model_cls.load_weights('../input/model-cls-0975acc/model_cls_1023_1053.h5')

def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0]
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
import segmentation_models as sm
# model_seg = sm.FPN('efficientnetb5', input_shape=(256, 1600, 3), encoder_weights=None,classes=4, activation='sigmoid')
# model_seg.load_weights('/kaggle/input/model-final-fpn-efnb5-bn-fone7684/model_final_fpn_efnb5_bn_FONE7684.h5')
model_seg = sm.FPN('efficientnetb3', input_shape=(256, 320, 3) ,encoder_weights=None,classes=4, activation='sigmoid')
model_seg.load_weights('../input/model-final-fpn-efnb3-bn-crop320-f1-7296/model_final_fpn_efnb3_bn_crop320_f1_7296.h5')

In [ ]:
# # 一般版本
# from tqdm import tqdm_notebook

# os.chdir('/kaggle/input/severstal-steel-defect-detection/')
# submit = pd.read_csv('sample_submission.csv')
# submit['ImageId_ClassId_1'] = submit.ImageId_ClassId.apply(lambda t: t[:-2])
# submit.set_index('ImageId_ClassId_1',inplace=True)
# all_test_img = os.listdir('test_images/')
# ii=0
# iii=0
# iiii=0
# for file in tqdm_notebook(all_test_img):
#     img = cv2.imread('test_images/'+file)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#     img_r = cv2.resize(img, (800, 128), interpolation=cv2.INTER_CUBIC)
#     img_r = np.expand_dims(efn.preprocess_input(img_r),0)
#     pred_cls = np.squeeze(model_cls.predict(img_r))
#     c = 0.5
# #     c1 = [7.59953548e-04, 1.62100789e-04, 2.55071200e-02, 5.34855464e-03]
#     c1 = [0.5,0.5,0.5,0.5]
#     if (pred_cls<c).all():
#         for i in range(4):
#             submit.loc[file,'EncodedPixels'].iloc[i]=''
#             ii+=1
#     else:
#         img = np.expand_dims(efn.preprocess_input(img),0)
#         pred_mask = model_seg.predict(img)
#         for i in range(4):
#             if pred_cls[i]<c:
#                 submit.loc[file,'EncodedPixels'].iloc[i]=''
#                 iii+=1
#             else:
#                 mask = np.squeeze(pred_mask[:,:,:,i])
# #                 print(mask.shape)
#                 mask[mask>c1[i]]=1
#                 mask[mask<c1[i]]=0
#                 submit.loc[file,'EncodedPixels'].iloc[i]=mask2rle(mask)
#                 iiii+=1

                
# submit[['ImageId_ClassId', 'EncodedPixels']].to_csv('/kaggle/working//submission.csv', index=False)
# print(ii+iii+iiii,ii,iii,iiii,ii+iii+iiii-sum(submit.EncodedPixels==''))

In [ ]:
# # 一般版本 TTA
# from tqdm import tqdm_notebook

# os.chdir('/kaggle/input/severstal-steel-defect-detection/')
# submit = pd.read_csv('sample_submission.csv')
# submit['ImageId_ClassId_1'] = submit.ImageId_ClassId.apply(lambda t: t[:-2])
# submit.set_index('ImageId_ClassId_1',inplace=True)
# all_test_img = os.listdir('test_images/')
# ii=0
# iii=0
# iiii=0
# for file in tqdm_notebook(all_test_img):
#     img = cv2.imread('test_images/'+file)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#     img_r = cv2.resize(img, (800, 128), interpolation=cv2.INTER_CUBIC)
# #     img_r = np.expand_dims(efn.preprocess_input(img_r),0)
    
#     # TTA 計算分類
    
#     ia = np.r_[np.expand_dims(efn.preprocess_input(img_r),0),
#                np.expand_dims(efn.preprocess_input(np.rot90(img_r,2)),0),
#                np.expand_dims(efn.preprocess_input(np.fliplr(img_r)),0),
#                np.expand_dims(efn.preprocess_input(np.rot90(np.fliplr(img_r),2)),0)]
#     pred_cls = np.squeeze(model_cls.predict(ia))
#     pred_cls = pred_cls.sum(axis=0)/4
    
# #     pred_cls = np.zeros(4)
# #     pred_cls += np.squeeze(model_cls.predict(np.expand_dims(efn.preprocess_input(img_r),0)))
# #     pred_cls += np.squeeze(model_cls.predict(np.expand_dims(efn.preprocess_input(np.rot90(img_r,2)),0)))
# #     pred_cls += np.squeeze(model_cls.predict(np.expand_dims(efn.preprocess_input(np.fliplr(img_r)),0)))
# #     pred_cls += np.squeeze(model_cls.predict(np.expand_dims(efn.preprocess_input(np.rot90(np.fliplr(img_r),2)),0)))
# #     pred_cls/=4

#     print(pred_cls)
    
#     c = 0.5
# #     c1 = [7.59953548e-04, 1.62100789e-04, 2.55071200e-02, 5.34855464e-03]
#     c1 = [0.5,0.5,0.5,0.5]
#     if (pred_cls<c).all():
#         for i in range(4):
#             submit.loc[file,'EncodedPixels'].iloc[i]=''
#             ii+=1
#     else:
# #         img = np.expand_dims(efn.preprocess_input(img),0)
        
#         ia_seg = np.r_[np.expand_dims(efn.preprocess_input(img),0),
#                    np.expand_dims(efn.preprocess_input(np.rot90(img,2)),0),
#                    np.expand_dims(efn.preprocess_input(np.fliplr(img)),0),
#                    np.expand_dims(efn.preprocess_input(np.rot90(np.fliplr(img),2)),0)]        
        
#         pred_mask = model_seg.predict(ia_seg)
#         for i in range(4):
#             if pred_cls[i]<c:
#                 submit.loc[file,'EncodedPixels'].iloc[i]=''
#                 iii+=1
#             else:
#                 mask0 = np.squeeze(pred_mask[0,:,:,i])
#                 mask1 = np.rot90(np.squeeze(pred_mask[1,:,:,i]),2)
#                 mask2 = np.fliplr(np.squeeze(pred_mask[2,:,:,i])) 
#                 mask3 = np.rot90(np.fliplr(np.squeeze(pred_mask[3,:,:,i])),2) 
                
#                 mask = (mask0+mask1+mask2+mask3)/4
                
# #                 mask = np.squeeze(pred_mask[:,:,:,i])
# #                 print(mask.shape)
#                 mask[mask>=c1[i]]=1
#                 mask[mask<c1[i]]=0
#                 submit.loc[file,'EncodedPixels'].iloc[i]=mask2rle(mask)
#                 iiii+=1

                
# submit[['ImageId_ClassId', 'EncodedPixels']].to_csv('/kaggle/working//submission.csv', index=False)
# print(ii+iii+iiii,ii,iii,iiii,ii+iii+iiii-sum(submit.EncodedPixels==''))

In [ ]:
# # 切五片版本 TTA
# from tqdm import tqdm_notebook

# os.chdir('/kaggle/input/severstal-steel-defect-detection/')
# submit = pd.read_csv('sample_submission.csv')
# submit['ImageId_ClassId_1'] = submit.ImageId_ClassId.apply(lambda t: t[:-2])
# submit.set_index('ImageId_ClassId_1',inplace=True)
# all_test_img = os.listdir('test_images/')
# ii=0
# iii=0
# iiii=0
# for file in tqdm_notebook(all_test_img):
#     img = cv2.imread('test_images/'+file)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#     img_r = cv2.resize(img, (800, 128), interpolation=cv2.INTER_CUBIC)
# #     img_r = np.expand_dims(efn.preprocess_input(img_r),0)
    
#     # TTA 計算分類
    
#     ia = np.r_[np.expand_dims(efn.preprocess_input(img_r),0),
#                np.expand_dims(efn.preprocess_input(np.rot90(img_r,2)),0),
#                np.expand_dims(efn.preprocess_input(np.fliplr(img_r)),0),
#                np.expand_dims(efn.preprocess_input(np.rot90(np.fliplr(img_r),2)),0)]
#     pred_cls = np.squeeze(model_cls.predict(ia))
#     pred_cls = pred_cls.sum(axis=0)/4
    
# #     pred_cls = np.zeros(4)
# #     pred_cls += np.squeeze(model_cls.predict(np.expand_dims(efn.preprocess_input(img_r),0)))
# #     pred_cls += np.squeeze(model_cls.predict(np.expand_dims(efn.preprocess_input(np.rot90(img_r,2)),0)))
# #     pred_cls += np.squeeze(model_cls.predict(np.expand_dims(efn.preprocess_input(np.fliplr(img_r)),0)))
# #     pred_cls += np.squeeze(model_cls.predict(np.expand_dims(efn.preprocess_input(np.rot90(np.fliplr(img_r),2)),0)))
# #     pred_cls/=4

#     print(pred_cls)
    
#     c = 0.5
# #     c1 = [7.59953548e-04, 1.62100789e-04, 2.55071200e-02, 5.34855464e-03]
#     c1 = [0.4,0.4,0.4,0.4]
#     if (pred_cls<c).all():
#         for i in range(4):
#             submit.loc[file,'EncodedPixels'].iloc[i]=''
#             ii+=1
#     else:
# #         img = np.expand_dims(efn.preprocess_input(img),0)
        
#         img_input = np.empty((20,256,320,3))
#         ia_seg = np.r_[np.expand_dims(efn.preprocess_input(img),0),
#                    np.expand_dims(efn.preprocess_input(np.rot90(img,2)),0),
#                    np.expand_dims(efn.preprocess_input(np.fliplr(img)),0),
#                    np.expand_dims(efn.preprocess_input(np.rot90(np.fliplr(img),2)),0)]    
#         for i in range(5):
#             img_input[i*4:(i+1)*4] = ia_seg[:,:,i*320:(i+1)*320,:]
        
#         pred_mask = model_seg.predict(img_input)
#         for i in range(4):
#             if pred_cls[i]<c:
#                 submit.loc[file,'EncodedPixels'].iloc[i]=''
#                 iii+=1
#             else:
#                 mask0 = np.c_[pred_mask[0,:,:,i],pred_mask[4,:,:,i],pred_mask[8,:,:,i],pred_mask[12,:,:,i],pred_mask[16,:,:,i]]
#                 mask1 = np.rot90(np.squeeze(np.c_[pred_mask[1,:,:,i],pred_mask[5,:,:,i],pred_mask[9,:,:,i],pred_mask[13,:,:,i],pred_mask[17,:,:,i]]),2)
#                 mask2 = np.fliplr(np.squeeze(np.c_[pred_mask[2,:,:,i],pred_mask[6,:,:,i],pred_mask[9,:,:,i],pred_mask[14,:,:,i],pred_mask[18,:,:,i]])) 
#                 mask3 = np.rot90(np.fliplr(np.squeeze(np.c_[pred_mask[3,:,:,i],pred_mask[7,:,:,i],pred_mask[11,:,:,i],pred_mask[15,:,:,i],pred_mask[19,:,:,i]])),2) 
                
# #                 mask0 = np.squeeze(pred_mask[0,:,:,i])
# #                 mask1 = np.rot90(np.squeeze(pred_mask[1,:,:,i]),2)
# #                 mask2 = np.fliplr(np.squeeze(pred_mask[2,:,:,i])) 
# #                 mask3 = np.rot90(np.fliplr(np.squeeze(pred_mask[3,:,:,i])),2) 
                
#                 mask = (mask0+mask1+mask2+mask3)/4
                
# #                 mask = np.squeeze(pred_mask[:,:,:,i])
# #                 print(mask.shape)
#                 mask[mask>=c1[i]]=1
#                 mask[mask<c1[i]]=0
#                 submit.loc[file,'EncodedPixels'].iloc[i]=mask2rle(mask)
#                 iiii+=1

                
# submit[['ImageId_ClassId', 'EncodedPixels']].to_csv('/kaggle/working//submission.csv', index=False)
# print(ii+iii+iiii,ii,iii,iiii,ii+iii+iiii-sum(submit.EncodedPixels==''))

In [ ]:
# 切五片版本 TTA ，用最大分類
from tqdm import tqdm_notebook

os.chdir('/kaggle/input/severstal-steel-defect-detection/')
submit = pd.read_csv('sample_submission.csv')
submit['ImageId_ClassId_1'] = submit.ImageId_ClassId.apply(lambda t: t[:-2])
submit.set_index('ImageId_ClassId_1',inplace=True)
all_test_img = os.listdir('test_images/')

for file in tqdm_notebook(all_test_img):
    img = cv2.imread('test_images/'+file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    c1 = [0.9,0.9,0.9,0.9]
        
    img_input = np.empty((20,256,320,3))
    ia_seg = np.r_[np.expand_dims(efn.preprocess_input(img),0),
               np.expand_dims(efn.preprocess_input(np.rot90(img,2)),0),
               np.expand_dims(efn.preprocess_input(np.fliplr(img)),0),
               np.expand_dims(efn.preprocess_input(np.rot90(np.fliplr(img),2)),0)]    
    for i in range(5):
        img_input[i*4:(i+1)*4] = ia_seg[:,:,i*320:(i+1)*320,:]

    pred_mask = model_seg.predict(img_input)
    for i in range(4):
        mask0 = np.c_[pred_mask[0,:,:,i],pred_mask[4,:,:,i],pred_mask[8,:,:,i],pred_mask[12,:,:,i],pred_mask[16,:,:,i]]
        mask1 = np.rot90(np.squeeze(np.c_[pred_mask[1,:,:,i],pred_mask[5,:,:,i],pred_mask[9,:,:,i],pred_mask[13,:,:,i],pred_mask[17,:,:,i]]),2)
        mask2 = np.fliplr(np.squeeze(np.c_[pred_mask[2,:,:,i],pred_mask[6,:,:,i],pred_mask[9,:,:,i],pred_mask[14,:,:,i],pred_mask[18,:,:,i]])) 
        mask3 = np.rot90(np.fliplr(np.squeeze(np.c_[pred_mask[3,:,:,i],pred_mask[7,:,:,i],pred_mask[11,:,:,i],pred_mask[15,:,:,i],pred_mask[19,:,:,i]])),2) 


        mask = (mask0+mask1+mask2+mask3)/4
        if mask.max()<c1[i]:
            submit.loc[file,'EncodedPixels'].iloc[i]=''
        else:
            mask[mask>=c1[i]]=1
            mask[mask<c1[i]]=0
            submit.loc[file,'EncodedPixels'].iloc[i]=mask2rle(mask)

                
submit[['ImageId_ClassId', 'EncodedPixels']].to_csv('/kaggle/working//submission.csv', index=False)
print(sum(submit.EncodedPixels==''))